In [163]:
import torch
import torch.nn as nn
from tensorflow.keras.layers import *
import librosa
import numpy as np

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
nn.Conv1d(in_channels, out_channels, kernel_size, stride=1)


nn.Conv1d(in_channels, out_channels, kernel_size, stride=1)


nn.Conv1d(in_channels, out_channels, kernel_size, stride=1)


Conv1d(16, 33, kernel_size=(3,), stride=(2,))

In [208]:
path = '/home/farisalasmary/Desktop/speech_vqa/SpeechVQA/OpenEnded_mscoco_val2014_questions/5546071.mp3'
path = '/home/farisalasmary/volume/arabic_kaldi_models/NEOM_30_sec.wav'
audio_signal, sr = librosa.load(path, sr=16_000)
audio_signal = audio_signal.reshape(1, -1)
audio_signal.shape
audio_signal = torch.tensor([audio_signal]*8)


(1, 480000)

In [209]:
# input size (N,C_in,L)
#  N is a batch size, C denotes a number of channels, L is a length of signal sequence.

audio_signal = torch.tensor([audio_signal]*8)
audio_signal.shape

torch.Size([8, 1, 480000])

In [210]:
test = nn.Sequential(
    nn.Conv1d(1, 32, 64, stride=2),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.MaxPool1d(4)
)

In [211]:
test(audio_signal).shape

torch.Size([8, 32, 59992])

In [212]:
#  N is a batch size, C denotes a number of channels, L is a length of signal sequence.
# input size (N,C_in,L)

In [213]:
# tf.keras.layers.Conv1D(filters, kernel_size, strides=1)
# nn.Conv1d(in_channels, out_channels, kernel_size, stride=1)

In [248]:
conv1_out_channels = 32
conv2_out_channels = 64
conv3_out_channels = 128
conv4_out_channels = 256
conv5_out_channels = 512

conv1_kernel_size = 64
conv2_kernel_size = 32
conv3_kernel_size = 16
conv4_kernel_size = 8
conv5_kernel_size = 4

conv1 = nn.Sequential(
    nn.Conv1d(1, conv1_out_channels, conv1_kernel_size, stride=2),
    nn.BatchNorm1d(conv1_out_channels),
    nn.ReLU(),
    nn.MaxPool1d(4)
)

conv2 = nn.Sequential(
    nn.Conv1d(conv1_out_channels, conv2_out_channels, conv2_kernel_size, stride=2),
    nn.BatchNorm1d(conv2_out_channels),
    nn.ReLU(),
    nn.MaxPool1d(4)
)


conv3 = nn.Sequential(
    nn.Conv1d(conv2_out_channels, conv3_out_channels, conv3_kernel_size, stride=2),
    nn.BatchNorm1d(conv3_out_channels),
    nn.ReLU(),
    nn.MaxPool1d(4)
)


conv4 = nn.Sequential(
    nn.Conv1d(conv3_out_channels, conv4_out_channels, conv4_kernel_size, stride=2),
    nn.BatchNorm1d(conv4_out_channels),
    nn.ReLU(),
    nn.MaxPool1d(4)
)

conv5 = nn.Sequential(
    nn.Conv1d(conv4_out_channels, conv5_out_channels, conv5_kernel_size, stride=2),
    # https://discuss.pytorch.org/t/error-expected-more-than-1-value-per-channel-when-training/26274/5
    nn.BatchNorm1d(conv5_out_channels), # this layer may result in an error if the input size is 1 example
    nn.ReLU()
)


In [249]:
import torch.nn.functional as F

class L2Norm(nn.Module):
    def __init__(self, dim):
        super(L2Norm, self).__init__()
        self.dim = dim
        
    def forward(self, x):
        #return F.normalize(x, p=2, dim=1)
        return F.normalize(x, p=2, dim=self.dim)



In [250]:
l2_norm = L2Norm(2)


In [251]:
model = nn.Sequential(
    conv1,
    conv2,
    conv3,
    conv4,
    conv5
)
model

Sequential(
  (0): Sequential(
    (0): Conv1d(1, 32, kernel_size=(64,), stride=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv1d(32, 64, kernel_size=(32,), stride=(2,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv1d(64, 128, kernel_size=(16,), stride=(2,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv1d(128, 256, kernel_size=(8,), stride=(2,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
 

In [258]:
out = model(audio_signal)
print(out.shape)
out = out.transpose(1, 2)
out.shape

torch.Size([8, 512, 57])


torch.Size([8, 57, 512])

In [253]:
# tmp1 = conv1(audio_signal)
# tmp2 = conv2(tmp1)
# tmp3 = conv3(tmp2)
# tmp4 = conv4(tmp3)
# tmp5 = conv5(tmp4)
# tmp5.shape

In [254]:
l2_norm(out)

tensor([[[0.0738, 0.0000, 0.0000,  ..., 0.0000, 0.0287, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.1039, 0.0000, 0.0657],
         [0.0292, 0.0000, 0.0729,  ..., 0.0864, 0.0000, 0.0995],
         ...,
         [0.0283, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0348, 0.0574, 0.1179,  ..., 0.0000, 0.0538, 0.0000],
         [0.0000, 0.0000, 0.0247,  ..., 0.1147, 0.0044, 0.0969]],

        [[0.0738, 0.0000, 0.0000,  ..., 0.0000, 0.0287, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.1039, 0.0000, 0.0657],
         [0.0292, 0.0000, 0.0729,  ..., 0.0864, 0.0000, 0.0995],
         ...,
         [0.0283, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0348, 0.0574, 0.1179,  ..., 0.0000, 0.0538, 0.0000],
         [0.0000, 0.0000, 0.0247,  ..., 0.1147, 0.0044, 0.0969]],

        [[0.0738, 0.0000, 0.0000,  ..., 0.0000, 0.0287, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.1039, 0.0000, 0.0657],
         [0.0292, 0.0000, 0.0729,  ..., 0.0864, 0.0000, 0.

In [207]:
speech_model = Sequential()
speech_model.add(Conv1D(32, 64, strides=2, input_shape=(None, 1), name='conv_1'))
speech_model.add(BatchNormalization())
speech_model.add(Activation('relu'))
speech_model.add(MaxPooling1D(pool_size=4))

speech_model.add(Conv1D(64, 32, strides=2, name='conv_2'))
speech_model.add(BatchNormalization())
speech_model.add(Activation('relu'))
speech_model.add(MaxPooling1D(pool_size=4))

speech_model.add(Conv1D(128, 16, strides=2, name='conv_3'))
speech_model.add(BatchNormalization())
speech_model.add(Activation('relu'))
speech_model.add(MaxPooling1D(pool_size=4))

speech_model.add(Conv1D(256, 8, strides=2, name='conv_4'))
speech_model.add(BatchNormalization())
speech_model.add(Activation('relu'))
speech_model.add(MaxPooling1D(pool_size=4))

speech_model.add(Conv1D(512, 4, strides=2, name='last_conv'))
speech_model.add(BatchNormalization())
speech_model.add(Activation('relu'))

'''
speech_model.add(LSTM(common_embedding_size, return_sequences=False))
speech_model.add(Lambda(l2_norm, output_shape=(common_embedding_size,)))
speech_model.add(Dense(common_embedding_size, activation=activation))
speech_model.add(Dropout(dropout))
'''

'\nspeech_model.add(LSTM(common_embedding_size, return_sequences=False))\nspeech_model.add(Lambda(l2_norm, output_shape=(common_embedding_size,)))\nspeech_model.add(Dense(common_embedding_size, activation=activation))\nspeech_model.add(Dropout(dropout))\n'

In [155]:
speech_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [159]:
speech_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv1D)              (None, None, 32)          2080      
_________________________________________________________________
batch_normalization (BatchNo (None, None, 32)          128       
_________________________________________________________________
activation (Activation)      (None, None, 32)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 32)          0         
_________________________________________________________________
conv_2 (Conv1D)              (None, None, 64)          65600     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 64)          256       
_________________________________________________________________
activation_1 (Activation)    (None, None, 64)          0

In [201]:
test2 = audio_signal[0].cpu().numpy().reshape(-1, 1, 1)
test2

array([[[-8.0110440e-06]],

       [[-1.8733368e-05]],

       [[ 3.2841867e-06]],

       ...,

       [[ 5.2101550e-05]],

       [[-5.4691573e-05]],

       [[ 0.0000000e+00]]], dtype=float32)

In [202]:
test2.shape

(22988, 1, 1)

In [203]:
audio_signal[0].shape

torch.Size([1, 22988])

In [204]:
speech_model.predict(test2)

ValueError: in user code:

    /home/farisalasmary/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/farisalasmary/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/farisalasmary/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/farisalasmary/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/farisalasmary/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/farisalasmary/anaco

In [263]:
common_embedding_size = 512
hidden_layer_size = common_embedding_size
lstm = nn.LSTM(common_embedding_size, hidden_layer_size, batch_first=True, num_layers=1)

In [264]:
lstm(out)[0].shape

torch.Size([8, 57, 512])

In [267]:
aaa = out.mean(axis=1)
aaa.shape

torch.Size([8, 512])

In [270]:
(aaa*aaa).shape

torch.Size([8, 512])

In [293]:
path = '/home/farisalasmary/volume/arabic_kaldi_models/NEOM_30_sec.wav'
def read_audios(audios_paths):
    if type(audios_paths) != list:
        raise ValueError('You MUST provide a list of paths of audio files....')
    
    audios_signals = []
    for audio_path in audios_paths:
        audio_signal, sr = librosa.load(audio_path, sr=16_000)
        audio_signal = audio_signal.reshape(1, -1)
        
        audios_signals.append(audio_signal)
        
    return torch.tensor(audios_signals)


In [294]:
read_audios([path]*8).shape

torch.Size([8, 1, 480000])

In [295]:
model(read_audios([path]*80)).shape

torch.Size([80, 512, 57])